# Using the Quantinuum Emulators via Azure Quantum and pytket-qsharp

This notebook contains examples for running quantum circuits on Quantinuum's emulators via `pytket-qsharp`.

An emulator can be used to get an idea of what a quantum device will output for our quantum circuit. This enables circuit debugging and optimization before running on a physical machine. Emulators differ from simulators in that they model the physical and noise model of the device whereas simulators may model noise parameters, but not physical parameters. The Quantinuum emulators run on a physical noise model of the Quantinuum H-Series devices. There are various noise/error parameters modeled. For detailed information on the noise model, see the *Quantinuum System Model H1 Emulator Product Data Sheet* at [Quantinuum H-series](https://www.quantinuum.com/products/h1).

There are a few options for using the emulators: 

1. **Basic Usage:** Use the emulator as provided, which represents both the physical operations in the device as well as the noise. This the most common and simplest way to use the emulator. 
2. **Noiseless Emulation:** Use the emulator without the physical noise model applied. The physical device operations are represented, but all errors are set to 0. 
3. **Noise Parameters (*advanced option*):** Experiment with the noise parameters in the emulator. There is no guarantee that results achieved changing these parameters will represent outputs from the actual quantum computer represented.
4. **Stabilizer Emulator:** Use of the emulator for circuits involving only Clifford operations. 

For more information, see the *Quantinuum System Model H1 Emulator Product Data Sheet* at [Quantinuum H-series](https://www.quantinuum.com/products/h1) for detailed information on each of the emulators available and the full list of options available.

**Emulator Usage:**
* [Basic Usage](#basic-usage)
* [Noiseless Emulation](#no-noise)
* [Noise Parameters (*advanced*)](#noise)
* [Stabilizer Emulator](#stabilizer)

## Installations <a class="anchor" id="installations"></a>

In order to run this notebook, you'll need to install a few packages into your local environment related to the [Microsoft QDK](https://learn.microsoft.com/en-us/azure/quantum/install-overview-qdk?tabs=tabid-vscode%2Ctabid-conda). For additional information or troubleshooting with the following installations, see: [Use Q# and Python with Jupyter Notebooks](https://learn.microsoft.com/en-us/azure/quantum/install-overview-qdk?tabs=tabid-vscode%2Ctabid-dotnetcli#use-q-and-python-with-jupyter-notebooks). If you prefer to use VS Code instead of Jupyter notebooks, see: [Using the Microsoft QDK with VS Code](https://learn.microsoft.com/en-us/azure/quantum/install-overview-qdk?tabs=tabid-vscode%2Ctabid-dotnetcli#use-q-and-python-with-visual-studio-and-visual-studio-code).

1. Install the [.NET SDK](https://dotnet.microsoft.com/en-us/download). Make note of what version is recommended with the Microsoft QDK here: [Use Q# and Python with Jupyter Notebooks](https://learn.microsoft.com/en-us/azure/quantum/install-overview-qdk?tabs=tabid-vscode%2Ctabid-dotnetcli#use-q-and-python-with-jupyter-notebooks).
1. Install the `qsharp` python package: `pip install qsharp`.
1. Run `dotnet tool install -g Microsoft.Quantum.IQSharp` in a CLI after .NET is installed.
1. Run `dotnet iqsharp install`.
1. Install the `azure-quantum` python package: `pip install azure-quantum`.
1. Install [pytket-qsharp](https://cqcl.github.io/pytket-qsharp/api/api.html): `pip install pytket-qsharp`.

## Emulator Usage

### Basic Usage <a class="anchor" id="basic-usage"></a>

This section covers usage of the emulator which represents a physical and noise model of the device being used. For example, if using the `quantinuum.sim.h1-1e` target, this emulates the H1-1 quantum computer.

Here the circuit is created via the pytket python library. For details on getting started with `pytket`, see pytket's [Getting Started](https://cqcl.github.io/tket/pytket/api/getting_started.html) page.

In [1]:
from pytket.circuit import Circuit
from pytket.circuit.display import render_circuit_jupyter

# Set up Bell Test
circuit = Circuit(2, name="Bell Test")
circuit.H(0)
circuit.CX(0, 1)
circuit.measure_all()

render_circuit_jupyter(circuit)

Select the emulation device. See the Azure Quantum [Quantinuum Provider](https://learn.microsoft.com/en-us/azure/quantum/provider-quantinuum) for information and target names for each of the H-Series systems available. The devices available to you will be listed under **Providers &rarr; Quantinuum** on the Azure portal.

You will find the Resource ID and Location in the **Overview** tab in the Azure portal quantum workspace.

Select a machine target and login to the Quantinuum API using your credentials. You will be prompted to login to your Azure Quantum account. For more information on the `AzureBackend`, see the pytket-qsharp [AzureBackend API](https://cqcl.github.io/pytket-qsharp/api/api.html#pytket.extensions.qsharp.AzureBackend).

In [2]:
from pytket.extensions.qsharp import AzureBackend

machine = 'quantinuum.sim.h1-2e'
resource_id = '/subscriptions/your-id'
location = 'your-location'
backend = AzureBackend(target_name=machine, resourceId=resource_id, location=location)

Connecting to Azure Quantum...

Connected to Azure Quantum workspace megan-qtm-test-aq in location eastus.
Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e


Compile the circuit to the Quantinuum backend with `get_compiled_circuit`. See the `pytket` [User Manual](https://cqcl.github.io/pytket/manual/index.html) for more information on all the options that are available.

In [3]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=1)

render_circuit_jupyter(compiled_circuit)

Run the circuit on the emulator chosen.

In [5]:
n_shots = 100
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=n_shots)
print(handle)

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: 50974a99-eff2-4bab-ba72-37f068759c57
('50974a99-eff2-4bab-ba72-37f068759c57', 100, 'null')


Check the job status.

In [6]:
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, message="{'id': '50974a99-eff2-4bab-ba72-37f068759c57', 'name': 'Bell Test', 'status': 'Waiting', 'uri': 'https://portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/f265e1a6-e359-4fca-842d-f90ecb910903/resourceGroups/QTM-Tests/providers/Microsoft.Quantum/Workspaces/megan-qtm-test-aq/job_management?microsoft_azure_quantum_jobid=50974a99-eff2-4bab-ba72-37f068759c57', 'provider': 'quantinuum', 'target': 'quantinuum.sim.h1-2e', 'creation_time': '2023-02-10T22:54:23.0154322+00:00', 'begin_execution_time': None, 'end_execution_time': None, 'cost_estimate': ''}", error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


Once a job's status returns completed, return results with the `get_result` function.

In [7]:
result = backend.get_result(handle)

result

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=Counter({OutcomeArray([[192]], dtype=uint8): 50, OutcomeArray([[0]], dtype=uint8): 49, OutcomeArray([[64]], dtype=uint8): 1}),shots=None,state=None,unitary=None,density_matrix=None)

Save jobs results.

In [ ]:
import json

with open('pytket_emulator_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

The result output is just like that of a quantum device. The simulation by default runs with noise.

In [8]:
result = backend.get_result(handle)
print(result.get_distribution())

{(0, 0): 0.49, (1, 1): 0.5, (0, 1): 0.01}


In [9]:
print(result.get_counts())

Counter({(1, 1): 50, (0, 0): 49, (0, 1): 1})


### Noiseless Emulation <a class="anchor" id="no-noise"></a>

The Quantinuum emulators may be run with or without the physical device's noise model. The default is the emulator runs with the physical noise model turned on. The physical noise model can be turned off by setting `noisy_simulation=False`.

In [10]:
n_shots = 100
no_error_model_handle = backend.process_circuit(compiled_circuit, 
                                                n_shots=n_shots,
                                                noisy_simulation=False)
print(no_error_model_handle)

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: 7b2f42c6-3378-43ce-a9be-63d1fcd72188
('7b2f42c6-3378-43ce-a9be-63d1fcd72188', 100, 'null')


In [12]:
no_error_model_status = backend.circuit_status(no_error_model_handle)
print(no_error_model_status)

CircuitStatus(status=<StatusEnum.SUBMITTED: 'Circuit has been submitted.'>, message="{'id': '7b2f42c6-3378-43ce-a9be-63d1fcd72188', 'name': 'Bell Test', 'status': 'Waiting', 'uri': 'https://portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/f265e1a6-e359-4fca-842d-f90ecb910903/resourceGroups/QTM-Tests/providers/Microsoft.Quantum/Workspaces/megan-qtm-test-aq/job_management?microsoft_azure_quantum_jobid=7b2f42c6-3378-43ce-a9be-63d1fcd72188', 'provider': 'quantinuum', 'target': 'quantinuum.sim.h1-2e', 'creation_time': '2023-02-10T22:55:16.8659409+00:00', 'begin_execution_time': None, 'end_execution_time': None, 'cost_estimate': ''}", error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [13]:
no_error_model_result = backend.get_result(no_error_model_handle)

no_error_model_result

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=Counter({OutcomeArray([[192]], dtype=uint8): 57, OutcomeArray([[0]], dtype=uint8): 41, OutcomeArray([[128]], dtype=uint8): 1, OutcomeArray([[64]], dtype=uint8): 1}),shots=None,state=None,unitary=None,density_matrix=None)

In [ ]:
with open('pytket_emulator_noiseless_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

In [14]:
no_error_model_result = backend.get_result(no_error_model_handle)
print(no_error_model_result.get_distribution())

print(no_error_model_result.get_counts())

{(1, 1): 0.57, (0, 0): 0.41, (1, 0): 0.01, (0, 1): 0.01}
Counter({(1, 1): 57, (0, 0): 41, (1, 0): 1, (0, 1): 1})


### Noise Parameters <a class="anchor" id="noise"></a>

The emulator runs with default error parameters that represent a noise environment similar to the physical devices. The `error-params` option can be used to override these error parameters and do finer-grain tweaks of the error model. For detailed information on the noise model, see the *Quantinuum System Model H1 Emulator Product Data Sheet* at [Quantinuum System Model H1](https://www.quantinuum.com/hardware/h1).

In this section, examples are given for experimenting with the noise and error parameters of the emulators. These are advanced options and not recommended to start with when doing initial experiments. As mentioned above, there is no guarantee that results achieved changing these parameters will represent outputs from the actual quantum computer represented. 

**Note**: All the noise parameters are used together any time a simulation is run. If only some of the parameters are specified, the rest of the parameters are used at their default settings. The parameters to override are specified with the `options` parameter.

* [Physical Noise](#physical-noise)
* [Dephasing Noise](#dephasing-noise)
* [Arbitrary Angle Noise Scaling](#arbitrary-angle-noise)
* [Scaling](#scaling)

#### Physical Noise <a class="anchor" id="physical-noise"></a>

See the *Quantinuum System Model H1 Emulator Product Data Sheet* at [Quantinuum System Model H1](https://www.quantinuum.com/hardware/h1) for information on these parameters.

In [16]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=100, 
                                 request_options={
                                     "options": {
                                         'error-params': {
                                             'p1': 4e-5,
                                             'p2': 3e-3,
                                             'p_meas': 3e-3,
                                             'p_init': 4e-5,
                                             'p_crosstalk_meas': 1e-5,
                                             'p_crosstalk_init': 3e-5,
                                             'p1_emission': 6e-6,
                                             'p2_emission': 2e-4
                                         }
                                     }})

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: a4c10a1e-1ecb-4320-ac69-04a4c06ec4fa


In [17]:
result = backend.get_result(handle)

print(result.get_distribution())

{(0, 0): 0.5, (1, 1): 0.5}


#### Dephasing Noise <a class="anchor" id="dephasing-noise"></a>

See the *Quantinuum System Model H1 Emulator Product Data Sheet* at [Quantinuum System Model H1](https://www.quantinuum.com/hardware/h1) for information on these parameters.

In [18]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=100, 
                                 request_options={
                                     "options": {
                                         'error-params': {
                                             'coherent_dephasing_rate': 0.2,
                                             'incoherent_dephasing_rate': 0.3,
                                             'coherent_dephasing': False,  # False => run the incoherent noise model
                                             'transport_dephasing': False, # False => turn off transport dephasing error
                                             'idle_dephasing': False # False => turn off idel dephasing error
                                         },
                                     }})

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: 2a1213ed-16c1-4fad-ae62-bba6dc26f679


In [19]:
result = backend.get_result(handle)

print(result.get_distribution())

{(0, 0): 0.37, (1, 1): 0.61, (1, 0): 0.02}


#### Arbitrary Angle Noise Scaling <a class="anchor" id="arbitrary-angle-noise"></a>

See the *Quantinuum System Model H1 Emulator Product Data Sheet* at [Quantinuum System Model H1](https://www.quantinuum.com/hardware/h1) for information on these parameters.

In [20]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=100, 
                                 request_options={
                                     "options": {
                                         'error-params': {
                                             'przz_a': 1.09,
                                             'przz_b': 0.051,
                                             'przz_c': 1.365,
                                             'przz_d': 0.035 
                                         },
                                     }})

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: bf638692-b4c8-47e8-baa0-5c2c50480593


In [21]:
result = backend.get_result(handle)

print(result.get_distribution())

{(1, 1): 0.5, (0, 0): 0.5}


#### Scaling <a class="anchor" id="scaling"></a>

All the error rates can be scaled linearly using the `scale` parameter. See the *Quantinuum System Model H1 Emulator Product Data Sheet* at [Quantinuum System Model H1](https://www.quantinuum.com/hardware/h1) for information on these parameters.

In [22]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=100, 
                                 request_options={
                                     "options": { 
                                         'error-params': {
                                             'scale': 0.1,  # scale error rates linearly by 0.1
                                         },
                                     }})

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: 284d7112-bea7-4662-ac2c-bd93a6902db7


In [23]:
result = backend.get_result(handle)

print(result.get_distribution())

{(1, 1): 0.44, (0, 0): 0.56}


Other aspects of the noise model can scale specific error rates in the error model, which are modeled here.

In [24]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=100, 
                                 request_options={
                                     "options": {
                                         'error-params': {
                                             'p1_scale': 0.1,
                                             'p2_scale': 0.1,
                                             'meas_scale': 0.1,
                                             'init_scale': 0.1,
                                             'memory_scale': 0.1,
                                             'emission_scale': 0.1,
                                             'crosstalk_scale': 0.1,
                                             'leakage_scale': 0.1
                                         },
                                     }})

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: 75c89b57-56b9-4b29-b466-edb6aadebcbd


In [25]:
result = backend.get_result(handle)

print(result.get_distribution())

{(1, 1): 0.5, (0, 0): 0.5}


### Stabilizer Emulator <a class="anchor" id="stabilizer"></a>

By default, emulations are run using a state-vector emulator, which simulates any quantum operation. However, if the quantum operations are all Clifford gates, it can be faster for complex circuits to use the `stabilizer` emulator.

In [28]:
n_shots = 100
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=n_shots,
                                 request_options={
                                     "options": {
                                         'simulator': 'stabilizer',
                                     }})
print(handle)

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: 2b171814-e634-40e9-b17f-75b50d7cbd59
('2b171814-e634-40e9-b17f-75b50d7cbd59', 100, 'null')


In [30]:
result = backend.get_result(handle)

result

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=Counter({OutcomeArray([[0]], dtype=uint8): 57, OutcomeArray([[192]], dtype=uint8): 43}),shots=None,state=None,unitary=None,density_matrix=None)

In [ ]:
with open('pytket_emulator_stabilizer_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

In [31]:
print(result.get_distribution())

print(result.get_counts())

{(1, 1): 0.43, (0, 0): 0.57}
Counter({(0, 0): 57, (1, 1): 43})


#### Noiseless Stabilizer

A noiseless stabilizer simulation can be specified via options in the `process_circuit` function with the following options:

- `simulator`: choose to run with a `stabilizer` simulator or `state-vector` (default is `state-vector`)
- `error-model`: whether to run with or without the physical device noise model on or off. The default is `True`, which means the physical noise model is turned on. If set to `False`, the physical noise model is turned off, performing noiseless simulation.

In [33]:
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=100, 
                                 request_options={
                                     "options": {
                                         'simulator': 'stabilizer',
                                         'error-model': False,
                                     }})

Loading package Microsoft.Quantum.Providers.Honeywell and dependencies...
Active target is now quantinuum.sim.h1-2e
Submitting TketCircuit to target quantinuum.sim.h1-2e...
Job successfully submitted.
   Job name: Bell Test
   Job ID: fb178a0c-c609-4944-af08-9658e175dc30


In [34]:
result = backend.get_result(handle)

print(result.get_distribution())

{(0, 0): 0.51, (1, 1): 0.48, (0, 1): 0.01}


<div align="center"> &copy; 2023 by Quantinuum. All Rights Reserved. </div>